In [1]:
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import datetime
import requests
from bs4 import BeautifulSoup as bs
import json

In [2]:
def get_etf():
    etf_csv=pd.read_csv('Developed Asia Pacific ETF List (108).csv') #讀資料夾底下的csv
    symbol=etf_csv['Symbol']#all etf
    inception=etf_csv['Inception'] # all etf inception time
    count=0
    real_symbol=[] #etf
    in_s=[] #time
    for candiate in inception:
        s=datetime.datetime.strptime(candiate, "%d/%m/%Y")
        if s<=datetime.datetime.strptime("31/12/2015", "%d/%m/%Y"):
           # print(s.strftime("%d/%m/%Y"))
            in_s.append(s.strftime("%d/%m/%Y"))
            real_symbol.append(symbol[count])
        count=count+1
    return real_symbol

In [3]:
def crawler(ETF,startDate,endDate,freq,target):
    dict={}
    date_arr=[]
    df = pd.DataFrame()
    page_url="https://query1.finance.yahoo.com/v8/finance/chart/"+ETF+"?symbol="+ETF+"&period1="+str(startDate)+"&period2="+str(endDate)+"&interval="+freq+"&includePrePost=true&events=div%7Csplit%7Cearn&lang=en-US&region=US&crumb=HLYrppuGGqQ&corsDomain=finance.yahoo.com&fbclid=IwAR298rNC7s-eA4POkUD5sVlKNEB2YuS4zL9WN23CPyO8CzyUYAF8ezIT9rM"
    r = requests.get(page_url)
    soup = BeautifulSoup(r.text, 'lxml')
    a=soup.p.get_text()
    python_dict = json.loads(a)
    if (python_dict['chart']['result'] == None):
        print("已下市")
    else:
        timestamp=python_dict['chart']['result'][0]['timestamp']
        for time in timestamp:
            date_arr.append(datetime.datetime.fromtimestamp(time).strftime("%Y-%m-%d"))
            dict['Date']=date_arr
        dict[ETF]=python_dict['chart']['result'][0]['indicators'][target][0][target]
        if(freq == '1wk' or freq == '1mo'):
            dict['Date'].pop() #移除最後一個
            dict[ETF].pop() #移除最後一個
        df = pd.DataFrame.from_dict(dict)
        df.set_index("Date" , inplace=True)
    return df

In [4]:
etfs=get_etf()
etfs.remove('ZJPN')
etfs.remove('PAF')# 移除不合規定的etf
now=datetime.datetime.now()
import time
now = int(time.mktime(now.timetuple()))
start_time=int(time.mktime(datetime.date(2016, 1, 1).timetuple()))



def get_Date(freq,target,etfs):
    print('----------'+freq+'----------')
    all_data = pd.DataFrame()
    for Symbol in etfs:
        print('start crawling :', Symbol)
        data = crawler(Symbol,start_time,now,freq,target)
        datalen = data.shape[0]
        if datalen == 0:
            continue
        if datalen > all_data.shape[0]:
            all_data = pd.concat([all_data, data], axis=1,join_axes=[data.index])
        else:
            all_data = pd.concat([all_data, data], axis=1,join_axes=[all_data.index])
    return all_data

In [5]:
all_data_week = get_Date('1wk','adjclose',etfs)
all_data_month = get_Date('1mo','adjclose',etfs)

----------1wk----------
start crawling : EWJ
start crawling : EWY
start crawling : FNDF
start crawling : VPL
start crawling : DXJ
start crawling : EWT
start crawling : EWH
start crawling : EPP
start crawling : FNDC
start crawling : DLS
start crawling : JPIN
start crawling : PXF
start crawling : GSIE
start crawling : EWA
start crawling : FDT
start crawling : IPAC
start crawling : GWX
start crawling : DWM
start crawling : DFJ
start crawling : HEWJ
start crawling : DBJP
start crawling : EWS
start crawling : DOL
start crawling : HFXI
start crawling : PDN
start crawling : IVLU
start crawling : IDOG
start crawling : DTH
start crawling : DIM
start crawling : DOO
start crawling : SCJ
start crawling : ENZL
start crawling : IDLB
start crawling : FJP
start crawling : JPXN
start crawling : IVAL
start crawling : DXJS
start crawling : ISCF
start crawling : DEEF
start crawling : IMOM
start crawling : JPN
start crawling : IDHQ
start crawling : FPA
start crawling : DVYA
start crawling : JPMV
start craw

In [18]:
all_data_week=all_data_week.dropna()
#all_data_week

,EWJ,EWY,FNDF,VPL,DXJ,EWT,EWH,EPP,FNDC,DLS,...,HFXJ,EZJ,ISZE,SCIJ,FHK,HJPX,SCIX,FKO,FAUS,HAUD
Date,,,,,,,,,,,,,,,,,,,,,
2015-12-28,46.485207,47.019604,21.969828,52.084366,46.555050,23.509829,17.884832,33.753376,24.937567,52.695538,...,17.614700,28.005575,20.226845,22.867174,30.752674,24.890738,17.958004,21.519485,24.272865,17.936274
2016-01-04,43.877121,44.236481,20.475660,48.692959,42.725044,21.576757,16.630545,30.605755,23.400639,49.629951,...,16.504934,25.051937,20.226845,22.001951,27.935358,23.312510,17.281570,20.922771,21.648773,18.888615
2016-01-11,42.879921,43.062653,19.866922,47.268383,41.432880,20.877169,16.016941,29.462763,22.566574,48.233196,...,16.011389,23.359154,20.226845,22.001951,26.837704,22.011570,17.830051,20.515495,21.251707,17.598196
2016-01-18,43.301815,44.795002,20.364979,48.104744,42.864483,21.742449,15.863537,30.368362,22.847719,48.614136,...,16.178421,23.921297,21.584774,21.437330,26.325462,21.489292,16.976858,19.947199,21.286236,17.598196
2016-01-25,44.107254,45.192593,20.835365,48.977871,44.212425,22.699778,16.215460,31.106913,23.475611,49.747856,...,16.095375,24.296061,20.226845,22.174980,26.352901,21.489292,17.170132,20.600740,23.176619,17.534975
2016-02-01,41.997772,45.003265,20.328087,47.204041,40.596226,22.810240,15.601852,30.341986,22.838350,48.713902,...,15.731117,22.438124,20.226845,21.882650,25.529661,23.322008,17.534048,21.623674,20.889170,17.407665
2016-02-08,39.926647,43.990360,19.737797,45.614029,37.007923,22.349985,15.646972,30.095806,22.182343,47.299015,...,14.323151,20.253069,20.226845,20.362732,25.557102,19.637585,16.889797,20.648096,21.674671,16.810085
2016-02-15,41.537521,44.141819,20.364979,47.047798,38.383751,23.307314,16.071083,31.106913,22.885206,48.768314,...,15.121502,21.872807,20.226845,21.391796,26.673052,21.213913,17.485292,20.903830,21.182653,17.538439
2016-02-22,42.419670,43.829426,20.420319,47.544102,39.796764,23.546646,16.350813,30.904696,23.156977,49.194595,...,15.608441,22.933577,20.226845,21.947309,26.801109,20.834076,17.695110,21.036432,22.045841,17.193750


In [7]:
#all_data_month

In [8]:
all_data_month=all_data_month.dropna()
#all_data_month

In [9]:
def  calculate_ra_1(etf): # calculate one ra
    today_etf=etf[-1]
    return [(today_etf-etf[i])/etf[i] for i in range(0,len(etf)-1)]

In [10]:
def calculate_ra_all(date,etfs,data):
    dict={}
    for etf in etfs:
        dict[etf]=calculate_ra_1(data[etf])
    
    df = pd.DataFrame.from_dict(dict)
    return df

In [19]:
returns=calculate_ra_all(all_data_month.index,etfs,all_data_month)

,EWJ,EWY,FNDF,VPL,DXJ,EWT,EWH,EPP,FNDC,DLS,...,HFXJ,EZJ,ISZE,SCIJ,FHK,HJPX,SCIX,FKO,FAUS,HAUD
0,0.240612,0.348672,0.323711,0.342096,0.144242,0.523363,0.616359,0.475556,0.323501,0.318936,...,0.263717,0.339626,0.293627,0.234725,0.446520,0.295901,0.439127,0.091797,0.347951,0.392645
1,0.310105,0.381962,0.360447,0.390904,0.307549,0.462855,0.605637,0.484789,0.343888,0.333523,...,0.303141,0.460890,0.293627,0.269124,0.419906,0.336656,0.395401,0.070148,0.419313,0.420284
2,0.250398,0.218963,0.263847,0.294016,0.245028,0.355200,0.469194,0.331086,0.237540,0.228245,...,0.250302,0.341204,0.224123,0.214279,0.331020,0.268436,0.284856,-0.026780,0.249740,0.370111
3,0.249303,0.232261,0.220017,0.285226,0.324092,0.438213,0.468451,0.306763,0.212645,0.214684,...,0.208859,0.360614,0.190641,0.204549,0.288806,0.312136,0.264252,-0.010869,0.250043,0.303601
4,0.210097,0.293923,0.238204,0.271714,0.239922,0.408026,0.470682,0.335854,0.196885,0.197177,...,0.278408,0.273852,0.190641,0.172146,0.291688,0.266792,0.291286,0.032422,0.266905,0.275300
5,0.240612,0.236520,0.259588,0.280614,0.403305,0.336873,0.483451,0.331765,0.228370,0.287856,...,0.324767,0.334567,0.190641,0.177610,0.264618,0.386452,0.254196,0.020244,0.210608,0.311849
6,0.180693,0.156766,0.217038,0.200844,0.321491,0.249705,0.362172,0.230051,0.163696,0.189694,...,0.266297,0.211078,0.188187,0.155923,0.182851,0.345240,0.169928,-0.008770,0.117978,0.253641
7,0.159541,0.142602,0.205265,0.198607,0.245933,0.249705,0.353752,0.260251,0.167384,0.187306,...,0.234654,0.173496,0.157034,0.141008,0.159113,0.373801,0.182898,-0.029369,0.127521,0.240864
8,0.135499,0.107614,0.186141,0.168335,0.254354,0.196374,0.293476,0.214050,0.131912,0.156545,...,0.206341,0.137671,0.153396,0.090517,0.121805,0.298945,0.184577,-0.048354,0.150065,0.258123
9,0.127408,0.163245,0.191815,0.175024,0.190736,0.191819,0.319861,0.251036,0.147584,0.173597,...,0.188978,0.135151,0.182714,0.094128,0.160839,0.237591,0.227615,0.023822,0.184101,0.245148


In [21]:
week_returns=calculate_ra_all(all_data_week.index,etfs,all_data_week)
#week_returns

0.052407182710593296

In [13]:
# returns=calculate_ra_all(all_data_month.index,etfs)
# returns

In [14]:
def OmegaSharpeRatio(R, L):
    L=L/100
    n = len(R)
    upside = 0.0
    downside = 0.0
    
    for i in range(n):
        if R[i] < L:
            downside = downside + (L-R[i])
        else:
            upside = upside + (R[i] - L)
            
    if downside != 0:
        OmegaRatio = (upside / downside)-1
    else:
        OmegaRatio = "undefined"
        
    return OmegaRatio

In [15]:
def ranking(etfs,returns,date):
    arr=[]
    symbols=[]
    if date=='week':
        rf=2.24/4
    else:
        rf=2.24
    for etf in etfs:
        if OmegaSharpeRatio(returns[etf],rf)!="undefined":
            arr.append(OmegaSharpeRatio(returns[etf],rf))
            symbols.append(etf)
    dict_omega={
        "ETF_ Symbol":symbols,
        "Value":arr
    }

    df=pd.DataFrame(dict_omega)
    df=df.sort_values(by=['Value'],ascending=False).set_index([pd.Series([i for i in range(1,len(df['Value'])+1)])])
    df.index.name="Ranking"
    return df

In [16]:
data=ranking(etfs,week_returns,'week')
data

,ETF_ Symbol,Value
Ranking,,
1,HAUD,2883.096131
2,EWH,817.184182
3,EPP,74.793238
4,SCIX,29.360763
5,EWA,25.142955
6,DOO,21.003610
7,IDHQ,13.957150
8,HFXI,13.154458
9,FAUS,12.799746


In [22]:
data=ranking(etfs,returns,'month')
data

,ETF_ Symbol,Value
Ranking,,
1,EWH,420.559737
2,HAUD,315.560366
3,EPP,44.908598
4,SCIX,26.698052
5,EWA,14.572445
6,DOO,5.288793
7,EWT,4.671522
8,FAUS,4.328689
9,AUSE,4.261590


In [36]:
import numpy as np
from numpy import mean
from numpy import var
from scipy.stats import kurtosis
from scipy.stats import skew
import math
import cmath

def sharpeASKSR(etf, returns, rf):
    x = np.asarray(returns[etf])
    u = mean(x)/x.size    #期望值
    v = var(x, ddof=1)    #變異數
    SD = math.sqrt(v)     #標準差
    S = skew(x)           #偏度
    K = kurtosis(x)       #峰度

    c1 = 3*K-4*pow(S,2)-9
    c2 = 3*K-5*pow(S,2)-9
    a = ( 3*(cmath.sqrt(c1) ) / ( v*c2 ) )
    b = ( 3*S ) / ( SD*c2 )
    n = u - ( (3*S*SD) / c1 )
    d = 3*SD * ( (cmath.sqrt(c2)) / (c1) ) 
    phi = cmath.sqrt(pow(a,2)*pow(b,2))
    a_ = b + ( a*(n-rf) / cmath.sqrt(pow(d,2)+pow(n-rf,2))) 
    ASKSR = cmath.sqrt(2*(a_*(n-rf)-d*(phi-cmath.sqrt(pow(a,2)-pow(b-a_,2)))))
    
    return ASKSR

In [40]:
def rankingByASKSR(etfs, returns):
    arr=[]
    symbols=[]
    for etf in etfs:
        if sharpeASKSR(etf, returns ,0)!="undefined":
            arr.append(sharpeASKSR(etf, returns ,0))
            symbols.append(etf)
    dict_ASKSR={
        "ETF_ Symbol":symbols,
        "Value":arr
    }

    df=pd.DataFrame(dict_ASKSR)
    df=df.sort_values(by=['Value'],ascending=False).set_index([pd.Series([i for i in range(1,len(df['Value'])+1)])])
    df.index.name="Ranking"
    return df
        

In [41]:
rankingByASKSR(etfs, returns)

,ETF_ Symbol,Value
Ranking,,
1,FKO,(2.9498890740303567+0j)
2,FPA,(2.6899085797757176+0j)
3,FDTS,(2.653108995005184+0j)
4,IMOM,(2.6519112166076875+0j)
5,FJP,(2.5174669760952977+0j)
6,EWV,(2.4241591905618622+0j)
7,ISCF,(2.258940534511312+0j)
8,DEEF,(2.2105180412140766+0j)
9,FDT,(2.1988967173805807+0j)
